# 2022 Flatiron Machine Learning x Science Summer School

## Step 8: Run symbolic regression on bottleneck DSN predictions

In this step, we perform symbolic regression on the latent feature and target predictions of bottleneck DSNs.

In [1]:
import os
import numpy as np
import joblib
from pysr import PySRRegressor
import sympy
from scipy import optimize

import warnings
warnings.filterwarnings("ignore")

### Step 8.1: Load data

We load the input data as well as the latent feature and target prediction data.

In [2]:
data_path = "data_1k"
data_ext = ".gz"

In [3]:
data = {}
for file_name in os.listdir(data_path):
    if file_name[-len(data_ext):] == data_ext:
        
        var = file_name[:-len(data_ext)]
        var_data = np.loadtxt(os.path.join(data_path, file_name))
        if len(var_data.shape) == 1:
            var_data = var_data.reshape(-1,1)
        data[var] = var_data

        print(f"Loaded {var} data.")

Loaded F00 data.
Loaded F00_p1 data.
Loaded F01 data.
Loaded F02 data.
Loaded F03 data.
Loaded F04 data.
Loaded F05 data.
Loaded F06 data.
Loaded F07 data.
Loaded F07_p1 data.
Loaded F08 data.
Loaded F09 data.
Loaded F11 data.
Loaded F11_p1 data.
Loaded G00 data.
Loaded G00_p1 data.
Loaded G01 data.
Loaded G02 data.
Loaded G03 data.
Loaded G04 data.
Loaded G05 data.
Loaded G06 data.
Loaded G07 data.
Loaded G07_p1 data.
Loaded G08 data.
Loaded G09 data.
Loaded G11 data.
Loaded G11_p1 data.
Loaded X00 data.
Loaded X01 data.
Loaded X02 data.
Loaded X03 data.
Loaded X04 data.
Loaded X05 data.
Loaded X06 data.
Loaded X07 data.
Loaded X08 data.
Loaded X09 data.
Loaded X10 data.
Loaded X10_100 data.
Loaded X11 data.
Loaded X11_100 data.
Loaded X11_1000 data.
Loaded X11_10000 data.
Loaded X11_std0100 data.
Loaded X11_std0125 data.
Loaded X11_std0200 data.
Loaded X11_std0300 data.
Loaded X11_std1000 data.


In [4]:
mask_ext = ".mask"

masks = {}
for var in data:
    if var[0] == "X":
        try:
            masks[var] = joblib.load(os.path.join(data_path, var + mask_ext))
            print(f"Loaded masks for {var} data.")
        except:
            print(f"No masks found for {var} data.")

Loaded masks for X00 data.
Loaded masks for X01 data.
Loaded masks for X02 data.
Loaded masks for X03 data.
Loaded masks for X04 data.
Loaded masks for X05 data.
Loaded masks for X06 data.
Loaded masks for X07 data.
Loaded masks for X08 data.
Loaded masks for X09 data.
Loaded masks for X10 data.
No masks found for X10_100 data.
No masks found for X11 data.
No masks found for X11_100 data.
No masks found for X11_1000 data.
No masks found for X11_10000 data.
No masks found for X11_std0100 data.
No masks found for X11_std0125 data.
No masks found for X11_std0200 data.
No masks found for X11_std0300 data.
No masks found for X11_std1000 data.


### Step 8.2: Run PySR

In [5]:
def get_model(X, y):

    model = PySRRegressor(
        procs=4,
        populations=30,
        niterations=30,
        maxsize=20,
        binary_operators=["plus", "sub", "mult"],
        unary_operators=["sin", "cos"], # "exp", "log_abs"      
        model_selection="best",
        verbosity=0
    )

    model.fit(X, y)

    return model

In [6]:
model_path = "models"
model_name = "pysr_models_1k_preds.pkl"

try:
    models = joblib.load(os.path.join(model_path, model_name))
except:
    models = {}

In [ ]:
# vars = sorted([k for k in data.keys() if k[0] == "F" and k not in models])
vars = ["F11_p1"]

for var in vars:

    # get target dimensions
    f_dim = data[var].shape[1]

    # get input variables
    g_var = "G" + var[1:]
    x_var = "X" + var[1:].split('_')[0]

    # get training mask
    try:
        mask = masks[x_var]["train"]
    except:
        mask = np.full(data[x_var].shape[0], True)

    models[var] = {g_var: [], x_var: []}
    for i in range(f_dim):

        # get target data
        y = data[var][mask,i]

        # learn f(x)
        print(f"Learning {var}_{i}({g_var}).")
        X = data[g_var][mask]
        models[var][g_var].append(get_model(X, y))

        joblib.dump(models, os.path.join(model_path, model_name))
    
        # learn f(g(x))
        print(f"Learning {var}_{i}({x_var}).")
        X = data[x_var][mask]
        models[var][x_var].append(get_model(X, y))

        joblib.dump(models, os.path.join(model_path, model_name))

    # get target dimensions
    g_dim = data[g_var].shape[1]

    models[g_var] = {x_var: []}
    for i in range(g_dim):

        # get target data
        y = data[g_var][mask,i]
   
        # learn g(x)
        print(f"Learning {g_var}_{i}({x_var}).")
        X = data[x_var][mask]
        models[g_var][x_var].append(get_model(X, y))

        joblib.dump(models, os.path.join(model_path, model_name))

In [ ]:
del_name = "hall_of_fame_"

for f in os.listdir():
    if del_name in f:
        os.remove(f)
        print(f"Deleted {f}.")

### Step 8.3: Check discovery

In [ ]:
disc_eps = 1e-3

In [ ]:
for d_var in models:
    for i_var in models[d_var]:
        for m, model in enumerate(models[d_var][i_var]):
            best = model.get_best()
            print(f"{d_var}_{m}({i_var}): {best.loss:.2e} - [{(' ','X')[best.loss < disc_eps]}] - {best.equation}")

### Step 8.4: Get full model and optimize constants

In [ ]:
d_var = "F11_p1"
l_var = "G11_p1"
i_var = "X11"

In [ ]:
def num2symbols(expr):
    
    w = sympy.Wild("w", properties=[lambda t: isinstance(t, sympy.Float)])
    n = expr.find(w)
    
    alphabet = [f"p{p}" for p in range(len(n))]
    s = sympy.symbols(" ".join(alphabet[:len(n)]))
    
    d = {k: v for k, v in zip(n, s)}
    
    return expr.subs(d), [float(k) for k in d]

def optimize_eq(eq, inits, in_data, target_data):

    i_syms = [f"x{i}" for i in range(in_data.shape[1])]
    
    def opt_fun(p):
        opt_eq = eq.subs({f"p{i}": p[i] for i in range(len(p))})
        return np.mean((sympy.lambdify(i_syms, opt_eq, modules='numpy')(*list(in_data.T)) - target_data)**2)
    
    res = optimize.minimize(opt_fun, inits, method="BFGS")
    opt_eq = eq.subs({f"p{i}": res.x[i] for i in range(len(res.x))})
        
    return opt_eq

In [ ]:
l_eqs = []
l_syms = []

for m, model in enumerate(models[l_var][i_var]):
    l_eqs.append(model.get_best().sympy_format)
    l_syms.append(sympy.Symbol(f'x{m}'))
    
    print(model.get_best().equation)

In [ ]:
f_eqs = []

for model in models[d_var][l_var]:
    f_eqs.append(model.get_best().sympy_format.subs(dict(zip(l_syms, l_eqs))))
    
print(f_eqs[0])

In [ ]:
print("2.7*x0**2 + 4.5*x0*x1 + 5.0*cos(3.0*x1)")

In [ ]:
for f, f_eq in enumerate(f_eqs):
    
    p_eq, inits = num2symbols(f_eq)
    print(p_eq)
    
    t_var = d_var.split('_')[0]
    o_eq = optimize_eq(p_eq, inits, data[i_var], data[t_var][:,f])
    print(o_eq)